In [2]:
using Pkg

Pkg.activate(".")

  Activating project at `~/Documents/Uni/Master/MA/visual_analysis`


In [3]:
using GLMakie
using GeoMakie
using EmpiricalOrthogonalFunctions
using NCDatasets 
using Dates
using BenchmarkTools

In [4]:
data_path = "/home/denis/workspace/data/ivt_fields_v1"

function get_files_of_member(scenario_id, member_nr)
    return readdir(joinpath(data_path, scenario_id, "r$(member_nr)i1p1f1"), join = true)    
end


get_files_of_member (generic function with 1 method)

In [5]:
function get_eof_of_datachunk(data)
    eof = EmpiricalOrthogonalFunction(data; timedim=3)

    temporalsignal = pcs(eof)
    spatialsignal = reshape(eofs(eof),(size(datain)[1:2]..., :))
    return temporalsignal, spatialsignal
end

get_eof_of_datachunk (generic function with 1 method)

In [6]:
function get_all_data(scenario_id, member_nr)

    file_paths = get_files_of_member(scenario_id, member_nr)

    time_data = Union{Missing, Dates.DateTime}[]
    ivt_data = Array{Union{Missing, Float64}, 3}[]

    

    for file_path in file_paths
        time_chunk, ivt_chunk = NCDataset(file_path) do ds
            return ds[:time][:]::Array{Union{Missing, Dates.DateTime}, 1}, ds[:ivt][:, :, :]::Array{Union{Missing, Float64}, 3}
        end
        append!(time_data, time_chunk)
        push!(ivt_data, ivt_chunk)
    end

    return time_data, cat(ivt_data..., dims=3)
end

get_all_data (generic function with 1 method)

In [7]:
time_data, ivt_data = get_all_data("ssp126", 10)

InterruptException: InterruptException:

In [8]:
time_data, ivt_data = NCDataset(get_files_of_member("ssp126", 10)[1]) do ds
    return ds[:time][:]::Array{Union{Missing, Dates.DateTime}, 1}, ds[:ivt][:, :, :]::Array{Union{Missing, Float64}, 3}
end

(Union{Missing, DateTime}[DateTime("2015-01-01T06:00:00"), DateTime("2015-01-01T12:00:00"), DateTime("2015-01-01T18:00:00"), DateTime("2015-01-02T00:00:00"), DateTime("2015-01-02T06:00:00"), DateTime("2015-01-02T12:00:00"), DateTime("2015-01-02T18:00:00"), DateTime("2015-01-03T00:00:00"), DateTime("2015-01-03T06:00:00"), DateTime("2015-01-03T12:00:00")  …  DateTime("2034-12-29T18:00:00"), DateTime("2034-12-30T00:00:00"), DateTime("2034-12-30T06:00:00"), DateTime("2034-12-30T12:00:00"), DateTime("2034-12-30T18:00:00"), DateTime("2034-12-31T00:00:00"), DateTime("2034-12-31T06:00:00"), DateTime("2034-12-31T12:00:00"), DateTime("2034-12-31T18:00:00"), DateTime("2035-01-01T00:00:00")], Union{Missing, Float64}[107.83024404217494 105.35183652372383 … 4.810704767513298 14.481626430503018; 138.11806601868335 132.92639636951492 … 5.865105532526301 14.261222253533777; … ; 32.42546716028079 93.37890870191428 … 7.1671847647811555 15.350266308663777; 35.96681207349521 100.77858958675463 … 6.00310975

In [9]:
size(ivt_data)

(70, 32, 29220)

In [10]:
function filter_by_date(fun, time_data, ivt_data, time_dim = 3)

    time_result = filter(fun, time_data)
    time_indices = [i for i in eachindex(time_data) if fun(time_data[i])]

    return time_result, ivt_data[:, :, time_indices]
end

filter_by_date (generic function with 2 methods)

In [11]:
winter_months = [12, 1, 2, 3]

4-element Vector{Int64}:
 12
  1
  2
  3

In [12]:
time_filtered, data_filtered = filter_by_date(time_data, ivt_data) do time_element
    

    for wm in winter_months
        if month(time_element) == wm
            return true
        end
    end
    return false
end

(Union{Missing, DateTime}[DateTime("2015-01-01T06:00:00"), DateTime("2015-01-01T12:00:00"), DateTime("2015-01-01T18:00:00"), DateTime("2015-01-02T00:00:00"), DateTime("2015-01-02T06:00:00"), DateTime("2015-01-02T12:00:00"), DateTime("2015-01-02T18:00:00"), DateTime("2015-01-03T00:00:00"), DateTime("2015-01-03T06:00:00"), DateTime("2015-01-03T12:00:00")  …  DateTime("2034-12-29T18:00:00"), DateTime("2034-12-30T00:00:00"), DateTime("2034-12-30T06:00:00"), DateTime("2034-12-30T12:00:00"), DateTime("2034-12-30T18:00:00"), DateTime("2034-12-31T00:00:00"), DateTime("2034-12-31T06:00:00"), DateTime("2034-12-31T12:00:00"), DateTime("2034-12-31T18:00:00"), DateTime("2035-01-01T00:00:00")], Union{Missing, Float64}[107.83024404217494 105.35183652372383 … 4.810704767513298 14.481626430503018; 138.11806601868335 132.92639636951492 … 5.865105532526301 14.261222253533777; … ; 32.42546716028079 93.37890870191428 … 7.1671847647811555 15.350266308663777; 35.96681207349521 100.77858958675463 … 6.00310975

In [21]:
years = Dict()

for d in time_filtered
    key = year(d)
    if haskey(years, key)
        push!(years[key], d)
    else
        years[key] = [d]
    end
end 
years[2017]

484-element Vector{DateTime}:
 2017-01-01T00:00:00
 2017-01-01T06:00:00
 2017-01-01T12:00:00
 2017-01-01T18:00:00
 2017-01-02T00:00:00
 2017-01-02T06:00:00
 2017-01-02T12:00:00
 2017-01-02T18:00:00
 2017-01-03T00:00:00
 2017-01-03T06:00:00
 ⋮
 2017-12-29T18:00:00
 2017-12-30T00:00:00
 2017-12-30T06:00:00
 2017-12-30T12:00:00
 2017-12-30T18:00:00
 2017-12-31T00:00:00
 2017-12-31T06:00:00
 2017-12-31T12:00:00
 2017-12-31T18:00:00

In [34]:

# create geographical axes
function local_geoaxis_creation!(
    figure::Makie.Figure,
    lonlims::Tuple{<:Number, <:Number}, 
    latlims::Tuple{<:Number, <:Number}; 
    lonpadding::Float64 = 1.0, 
    latpadding::Float64 = 1.0,
    figure_x::Int = 1,
    figure_y::Int = 1,
    title::String = ""
    )
    
    paddingfun = (x, y) -> x < 0 ? x-y : x + y

    lon_padded = paddingfun.(lonlims, lonpadding)
    lon_center = lon_padded[1] + ((lon_padded[2] - lon_padded[1]) / 2)



    lat_padded = paddingfun.(latlims, latpadding)
    lat_center = lat_padded[1] + ((lat_padded[2] - lat_padded[1]) / 2)
    
    geoaxis = GeoMakie.GeoAxis(
        figure[figure_x, figure_y]; 
        dest = "+proj=merc +lon_0=$(lon_center) +lat_0=$(lat_center)",
        # source = dest,
        # lonlims = lonlims,
        # latlims = latlims,
        limits=(lonlims, latlims),
        title = title
    )

    

    return geoaxis
end


function build_figure(
    data,
    lon_bounds::Tuple{<:Number, <:Number},
    lat_bounds::Tuple{<:Number, <:Number};
    title::String = "",
    lonpadding::Float64 = 1.0, 
    latpadding::Float64 = 1.0,
    colormap = :viridis,
    colorrange::Union{Nothing, Tuple{<:Real, <:Real}} = nothing, 
    shading = Makie.automatic,
    resolution::Union{Nothing, Tuple{Int, Int}})::Makie.Figure

    fig = isnothing(resolution) ?  Figure(fontsize=12) : Figure(size = resolution, fontsize=12)

    

    ga = local_geoaxis_creation!(fig, lon_bounds, lat_bounds; lonpadding, latpadding, title=title)
    surface!(ga, lon_bounds[1]..lon_bounds[2], lat_bounds[1]..lat_bounds[2], data; shading = shading, colormap = colormap)
    lines!(ga, GeoMakie.coastlines(); color = :red, transformation = (; translation = (0, 0, 1000)))
    return fig
end

build_figure (generic function with 1 method)

In [36]:
build_figure(data_filtered[:, :, 1], (-90, 40), (20, 80); title = "Test IVT 1", resolution = (1000, 1000), colormap = :redsblues)